# Automated CMEMS downloader for all operating systems

This routine contains an examples of how to use python to set up an automated downloader of CMEMS data.

    Version: 1.1
    Author: B loveday, PML
    Notes:
    1. The python-motu client must be installed
      

In [1]:
#!/usr/bin/env python

#-imports-----------------------------------------------------------------------
import os, sys, shutil
import argparse
import logging
import datetime
import subprocess

In [2]:
#-functions---------------------------------------------------------------------
def download_data(Command, logging, verbose=False):
    
    processed_state = 'Downloaded ok'
    logging.info('Launching download CMD:  '+Command)

    try:
        process = subprocess.Popen(CMD, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
        process.wait()

        # Poll process for new output until finished
        while True:
            nextline = process.stdout.readline()
            if nextline == '' and process.poll() is not None:
                break
            if nextline !='':
                logging.info(nextline)
            if 'Error' in nextline:
                processed_state = nextline
            sys.stdout.flush()

        output   = process.communicate()[0]
        exitCode = process.returncode

        if (exitCode == 0):
            logging.info('Downloading successful')    
            processed_flag = True
        else:
            logging.error('Something went wrong in downloading: see above')
            processed_flag = False   
    except:
        logging.info('Downloading unsuccessful')    
        processed_flag = False
        processed_state = 'Unknown Error'

    return processed_flag, processed_state

In [3]:
#-default parameters------------------------------------------------------------
DEFAULT_LOG_PATH    = os.getcwd()

In [4]:
#-args--------------------------------------------------------------------------
#parser = argparse.ArgumentParser()
#args = parser.parse_args()

In [5]:
#-main--------------------------------------------------------------------------
if __name__ == "__main__":
    # preliminary stuff
    logfile = os.path.join(DEFAULT_LOG_PATH,"CMEMS_DOWNLOAD_"+datetime.datetime.now().strftime('%Y%m%d_%H%M')+".log")
    verbose=False
    
    # set file logger
    try:
        if os.path.exists(logfile):
            os.remove(logfile)
        print("logging to: "+logfile)
        logging.basicConfig(filename=logfile,level=logging.DEBUG)
    except:
        print("Failed to set logger")
    
    # set our variables
    motu_path  = os.getcwd()
    username   = 'your username'
    password   = 'your password'
    outdir     = os.path.join(motu_path,'Data')
    product_id = 'dataset-duacs-nrt-blacksea-merged-allsat-phy-l4-v3'
    service_id = 'SEALEVEL_BS_PHY_L4_NRT_OBSERVATIONS_008_041-TDS'
    date_min   = datetime.datetime(2018,2,3)
    date_max   = datetime.datetime(2018,2,4)
    lonmin     = 27.0625
    lonmax     = 41.9375
    latmin     = 40.0625
    latmax     = 46.9375
    variables  = ['sla','ugosa','vgosa']

    # clear the output directory and make a new one
    if os.path.exists(outdir):
        shutil.rmtree(outdir)
    os.mkdir(outdir)
    
    # set variables
    v_string=' --variable '
    all_variables = ' '
    for vv in variables:
        all_variables=v_string+"'"+vv+"'"+all_variables

    # loop through dates
    this_date = date_min
    while this_date <= date_max:
        date_format=this_date.strftime('%Y-%m-%d')
        outname = product_id+'_'+date_format+'.nc'
        print '---------------------'
        print('Saving to: '+outname)
        this_date = this_date + datetime.timedelta(days=1)
        CMD="python "+motu_path+"/motu-client-python/motu-client.py --user '"+username+"' --pwd '"+password+"' --motu 'http://motu.sltac.cls.fr/motu-web/Motu' --service-id "+service_id+" --product-id '"+product_id+"' --longitude-min '"+str(lonmin)+" ' --longitude-max '"+str(lonmax)+"' --latitude-min '"+str(latmin)+"' --latitude-max '"+str(latmax)+"' --date-min '"+date_format+"' --date-max '"+date_format+"' "+all_variables+" --out-dir '"+outdir+"' --out-name '"+outname+"'"
        if verbose:
            print CMD
        flag, state = download_data(CMD,logging)    


logging to: /Users/benloveday/Documents/Code/Autodownloader_CMEMS/CMEMS_DOWNLOAD_20180204_1105.log
---------------------
Saving to: dataset-duacs-nrt-blacksea-merged-allsat-phy-l4-v3_2018-02-03.nc
---------------------
Saving to: dataset-duacs-nrt-blacksea-merged-allsat-phy-l4-v3_2018-02-04.nc
